In [29]:
import pytesseract
import PIL.Image
import pandas as pd

In [30]:
myconfig = r"--psm 6 --oem 3"

In [31]:
text_data= pytesseract.image_to_data(PIL.Image.open('Adobe Scan 19-Jan-2024_2.jpg'),output_type='data.frame', config=myconfig)

In [32]:
text_data

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1530,2200,-1.000000,NaN
1,2,1,1,0,0,0,2,0,1528,2153,-1.000000,NaN
2,3,1,1,1,0,0,384,0,1146,26,-1.000000,NaN
3,4,1,1,1,1,0,384,0,1146,26,-1.000000,NaN
4,5,1,1,1,1,1,384,0,6,1,72.672157,2
...,...,...,...,...,...,...,...,...,...,...,...,...
198,5,1,1,6,13,1,69,2008,98,34,96.565804,MADE
199,5,1,1,6,13,2,180,2013,32,31,95.724571,IN
200,5,1,1,6,13,3,224,2014,170,37,95.724571,MALAYSIA
201,5,1,1,6,13,4,925,1899,247,231,68.407753,A\


In [33]:
text_data = text_data[text_data.conf != -1]

In [34]:
# Group by relevant columns (e.g., block_num, par_num, line_num)
lines = text_data.groupby(['block_num', 'par_num', 'line_num'])['text'].apply(lambda x: ' '.join(list(x))).tolist()
confs = text_data.groupby(['block_num', 'par_num', 'line_num'])['conf'].mean().tolist()

In [35]:
# Create a dictionary to store confidence scores for specific words and their variations
word_variations = {
    "Model": ['ModelNbr', 'model', 'modelno', 'model no', 'MODEL', 'MODEL NO','Model_No','Model'],
    "Serial": ['SerialNo', 'serial', 'serialnumber', 'serialno','Serial_No','Serial Number'],
    "Manufacturer": ['Manufacturer', 'manufacturer', 'make','brand','Brand']
}

In [36]:
# Extract confidence scores for each word and its variations
word_confidence = {}
for word, variations in word_variations.items():
    for line, conf in zip(lines, confs):
        for variation in variations:
            if variation.lower() in line.lower():
                word_confidence[word] = conf
                break

In [37]:
# Print confidence scores for each word and its variations
for word, conf in word_confidence.items():
    print(f"Confidence for '{word}': {conf}")

Confidence for 'Model': 77.9515325
Confidence for 'Serial': 92.17736057142858
Confidence for 'Manufacturer': 77.583404625


In [38]:
lines = []

# Iterate through the rows of the DataFrame
for index, row in text_data.iterrows():
    line_num = row['line_num']
    word = row['text']

    # If the lines list is empty or the current line_num is different from the previous one, start a new line
    if not lines or lines[-1]['line_num'] != line_num:
        lines.append({'line_num': line_num, 'text': word})
    else:
        # Append the word to the existing line
        lines[-1]['text'] += ' ' + word

# Combine the lines into a single variable while preserving the original sequence
result_text = '\n'.join(line['text'] for line in sorted(lines, key=lambda x: x['line_num']))

# Display or use the extracted text
print(result_text)

2 KGa BSE) Bee Gad AS a a Be EAE PO ae eee Fe ee Sct |HITACHI [== | H FACTORY CHARGE 1.500kg
_ [OPERATION MODE COOLING HEATING
MLTOTAL CURRENT (MAX. |_8.30-7.60A (12.0) | 8.40-7.74A ( 12.0A) | J RAI-E60YHA
ROOM AIR CONDITIONER
CAPACITY, |. NTT OW. Lo) ROO
OPERATION MODE | COOLING | HEATING.
HEAT PUMP OUTDOOR UNIT VOLTAGE 1 PHASE 220-240V “U
LTOTAL INPUT (MAX) "| T800W (2600) | 1830 (2650)
2 ee SL C0, ES SA
MODEL RAC-E60YHA | FREQUENCY 50/60Hz
OTAL INPUT (MAX 7 Te00W (26000) | T950Wv (26500)
SERIAL NO. TEST PRESSURE : HIGH 4.15MPa
TOTAL CURRENT [MAX] | 30-7.608 (120A) | 195-8.204 (120A)
1882483827 LOW 2.50MPa
INDOOR UNIT MODEL RAD-E60YHA
UNDOORUNITMODEL ss ts*=*SdY RAS-E60YHA
}OPERATION MODE COOUNG HEATING 7
12 A SE NER 5 AG ONG
TOTAL INPUT (MAX) 700w 001m) | —900w (oasomh
LTOTAL CURRENT (MAX) "| 7.80-7.15A( 120A) 8.70-8.00A (12.0A)
Manufacturer: Johnson Control: i Ai i IP X4
Pol aes 0, an Kenajan Bat dad oe
MADE IN MALAYSIA A\ Em
